https://developers.google.com/gmail/api/quickstart/python

In [1]:
!hostname

d8fd25e85ef6


# install modules

In [2]:
!pip install --upgrade google-api-python-client

    100% |████████████████████████████████| 61kB 349kB/s 
    100% |████████████████████████████████| 71kB 297kB/s 
    100% |████████████████████████████████| 92kB 677kB/s 
    100% |████████████████████████████████| 102kB 756kB/s 
    100% |████████████████████████████████| 163kB 189kB/s 
    100% |████████████████████████████████| 81kB 254kB/s 
  Running setup.py bdist_wheel for googleapis-common-protos ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/2c/f9/7f/6eb87e636072bf467e25348bbeb96849333e6a080dca78f706
Successfully built googleapis-common-protos


In [3]:
!pip install --upgrade oauth2client

    100% |████████████████████████████████| 102kB 228kB/s 


In [4]:
from __future__ import print_function
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

# authenticate our token

The following code cell intentionally prevents the rest of the notebook from automatically executing

In [5]:
stop -- action needed

SyntaxError: invalid syntax (<ipython-input-5-5214759fac44>, line 1)

the script below must be run as a .py file in order to launch your browser and authenticate

The .py file contains

If the error message is

`raise HttpAccessTokenRefreshError(error_msg, status=resp.status)
oauth2client.client.HttpAccessTokenRefreshError: invalid_grant: Bad Request`
        
then delete "token.json"

# now that we've authenticated with our token, we can send email
https://developers.google.com/gmail/api/guides/sending

## message text needs to be base64 encoded

https://en.wikipedia.org/wiki/Base64

"Base64 is a group of binary-to-text encoding schemes that represent binary data in an ASCII string format by translating it into a radix-64 representation."

Why do we use base64 for SMTP? Answer: SMTP is so old that the number of characters is stricted. To expand the character set, base64 encode. https://www.quora.com/Why-are-MIME-attachments-encoded-in-base64 and https://superuser.com/questions/402193/why-is-base64-needed-aka-why-cant-i-just-email-a-binary-file

In [6]:
import base64
base64.b64encode(bytes('your string', 'utf-8'))

b'eW91ciBzdHJpbmc='

a function that creates a base64 encoded email

In [7]:
#https://docs.python.org/3/library/email.mime.html
from email.mime.text import MIMEText
from email import encoders
    
def create_message(sender, to, subject, message_text):
    """Create a message for an email.

    Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

    Returns:
    An object containing a base64url encoded email object.
    """
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    
    # https://docs.python.org/3/library/base64.html
    return {'raw': base64.urlsafe_b64encode(message.as_string().encode('utf-8')).decode("utf-8")}
    # https://stackoverflow.com/questions/606191/convert-bytes-to-a-string
    
    #return {'raw': base64.urlsafe_b64encode(bytes("None",'utf-8'))}
    
    # https://docs.python.org/3.4/library/email-examples.html
    #return {'raw': encoders.encode_base64(message.as_string())} 

In [8]:
msg=create_message(sender="benpayne@umbc.edu", 
                   to="ben.is.located@gmail.com", 
                   subject="a subjct", 
                   message_text="msg text from Python")
msg

{'raw': 'Q29udGVudC1UeXBlOiB0ZXh0L3BsYWluOyBjaGFyc2V0PSJ1cy1hc2NpaSIKTUlNRS1WZXJzaW9uOiAxLjAKQ29udGVudC1UcmFuc2Zlci1FbmNvZGluZzogN2JpdAp0bzogYmVuLmlzLmxvY2F0ZWRAZ21haWwuY29tCmZyb206IGJlbnBheW5lQHVtYmMuZWR1CnN1YmplY3Q6IGEgc3ViamN0Cgptc2cgdGV4dCBmcm9tIFB5dGhvbg=='}

In [9]:
from googleapiclient import errors

def send_message(service, user_id, message):
    """Send an email message.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.

  Returns:
    Sent Message.
    """
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                   .execute())
        print ('Message Id: %s' % message['id'])
        return message
    # https://stackoverflow.com/questions/42541857/error-using-gmail-api-tuto-using-python-3-except-errors-httperror-error
    except errors.HttpError as error:
        print ('An error occurred: %s' % error)

In [10]:
store = file.Storage('token.json')
creds = store.get()
service = build('gmail', 'v1', http=creds.authorize(Http()))

In [11]:
send_message(service, "me", msg)

Message Id: 1708480c3b6a5022


{'id': '1708480c3b6a5022',
 'threadId': '1708480c3b6a5022',
 'labelIds': ['UNREAD', 'SENT', 'INBOX']}

The following also works